In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.datasets import mnist

(X_train, y_train), (X_test, y_test)= mnist.load_data()

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)


(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [ ]:
example_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)) #converting the data into tensors instead of numpy arrays

len(example_dataset) #all 60000 images

60000

In [ ]:
def normalize_img(image, label):
  return (tf.cast(image, tf.float32)/ 255.0, label) #normalizing the image by dividing by 255.0 so every pizel value is now in between 0 and 1, label remains unchanged

example_dataset= example_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

'''
Mappiing with parallel calls and autotune:
map applies the mormalize function to the entire dataset
and parallel calls uses tf autotune to find the optimum number for parallel calls, instead of setting a manual number of parallel calls
parallel calls ensure the cpu/gpu is constantly working, instead of waiting for data
autotune optimizes 'throuhgput': the gpu/cpu is not waiting for data
'''



"\nMappiing with parallel calls and autotune:\nmap applies the mormalize function to the entire dataset\nand parallel calls uses tf autotune to find the optimum number for parallel calls, instead of setting a manual number of parallel calls\nparallel calls ensure the cpu/gpu is constantly working, instead of waiting for data\nautotune optimizes 'throuhgput': the gpu/cpu is not waiting for data\n"

In [ ]:
example_dataset= example_dataset.cache() #storing the results of expensive operations so that they do not need to repeatedly run

example_dataset= example_dataset.shuffle(len(example_dataset))#shuffling the elements to prevent the model from recognizing or memorizing patters

example_dataset= example_dataset.batch(64)

example_dataset=example_dataset.prefetch(tf.data.AUTOTUNE)




Actual Code for the pipeline that we will use

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset=tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_dataset=train_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset=train_dataset.cache()

train_dataset=train_dataset.shuffle(len(train_dataset)) #reshuffle_each_iteration=True)
train_dataset=train_dataset.batch(64)
train_dataset=train_dataset.prefetch(tf.data.AUTOTUNE)




In [ ]:
test_dataset=tf.data.Dataset.from_tensor_slices((X_test, y_test))

test_dataset=test_dataset.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(64)
test_dataset= test_dataset.cache()

test_dataset=test_dataset.prefetch(tf.data.AUTOTUNE)



Creating the network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()

model.add(layers.InputLayer(input_shape=(28,28,1)))
#model.add(layers.Flatten())
model.add(layers.Conv2D(32,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(32,(3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 11, 11, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 32)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 800)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 61,482 (240.16 KB)

 Trainable params: 61,482 (240.16 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import EarlyStopping


es = EarlyStopping(patience=5) #prevents overfitting, stops the training if the models performance does not improve after 5 epochs


model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])



model.fit(
    train_dataset,
    epochs=50,
    validation_data=test_dataset,
    callbacks=[es]
)

Epoch 1/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.8658 - loss: 0.4651 - val_accuracy: 0.9814 - val_loss: 0.0610
Epoch 2/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9794 - loss: 0.0700 - val_accuracy: 0.9873 - val_loss: 0.0397
Epoch 3/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9867 - loss: 0.0457 - val_accuracy: 0.9879 - val_loss: 0.0356
Epoch 4/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9896 - loss: 0.0330 - val_accuracy: 0.9883 - val_loss: 0.0355
Epoch 5/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9910 - loss: 0.0280 - val_accuracy: 0.9886 - val_loss: 0.0335
Epoch 6/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9920 - loss: 0.0245 - val_accuracy: 0.9912 - val_loss: 0.0264
Epoch 7/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9946 - loss: 0.0163 - val_accuracy: 0.9883 - val_loss: 0.0401
Epoch 8/50
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9950 - loss: 0.0155 - val_accuracy: 0.